<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
<br>汉化的库: <a href="https://github.com/GoatCsu/CN-LLMs-from-scratch.git">https://github.com/GoatCsu/CN-LLMs-from-scratch.git</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# 第二章:处理文本数据 

本章节需要安装的包

pip3 install importlib.metadata

In [5]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))
# 确认库已安装并显示当前安装的版本

torch version: 2.5.1
tiktoken version: 0.8.0


- 本章节已经为LLM的实现构建了数据库

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/01.webp?timestamp=1" width="500px">

## 2.1 理解文字embedding

- 无代码

- 在众多形式的embedding中,我们只讨论text embedding
- embedding含义丰富,而且是常用词汇,所以以下皆不做翻译,多加体会!

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/02.webp" width="500px">

- LLM从高纬空间视角理解文字(i.e., 上千个dimension)
- 虽然人类只能想象低维的视角,所以我们无法描绘计算机所理解的embedding
- 但是下图我们粗浅的从二维上模拟计算机的视角

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/03.webp" width="300px">

## 2.2 文本标签化(tokenize)

- 关于embedding token,实在是不好翻译,于是有时候我会选择不去翻译这两个专有名词

- 本节中,我们将tokenize文字信息. 这会把文字拆解为更多小的理解单元 例如单个词或者字节

(这也有点抽象，事实上可以粗略理解为将单词拆分为词根、词源和词缀)

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/04.webp" width="300px">

- 载入源文件
- [The Verdict by Edith Wharton](https://en.wikisource.org/wiki/The_Verdict) 一本无版权的短篇小说

In [14]:
import os##导入os库    
import urllib.request ##导入request库

if not os.path.exists("the-verdict.txt"):##如果文件不存在则创建，防止因文件已存在而报错
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)##从指定的地点读取文件

- 如果在执行前面的代码单元时遇到 `ssl.SSLCertVerificationError` 错误可能是由于使用了过时的 Python 版本；
- 你可以在 [GitHub 上查阅更多信息](https://github.com/rasbt/LLMs-from-scratch/pull/403)。

In [15]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read() ##读入文件按照utf-8
    
print("Total number of character:", len(raw_text))##先输出总长度
print(raw_text[:99])##输出前一百个内容

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


- 目标是对这段文本进行分词和嵌入处理，以便用于大语言模型（LLM）。
- 我们将基于一些简单的示例文本开发一个简单的分词器，之后可以将其应用于上述文本。
- 以下正则表达式将基于空格进行分割。

In [16]:
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)##正则表达式按照空白字符进行分割

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


- 优化正则表达,可以分割更多的符号

In [17]:
result = re.split(r'([,.]|\s)', text)##只是按照, .分割
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


- 移除空格

In [18]:
##把上述结果去掉空格
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


- 我们还需要处理其他标点符号

In [19]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text) ##就是按照常用的符号分割
result = [item.strip() for item in result if item.strip()]##去掉两端的空白字符 也是去掉了空字符串与仅包含空白字符的项
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


- 万事俱备，我们现在来看一下文字处理后的效果

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/05.webp" width="350px">

In [20]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text) ##按照符号继续把raw给分割了
preprocessed = [item.strip() for item in preprocessed if item.strip()]##去掉两端的空白字符 也是去掉了空字符串喝仅包含空白字符的项
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


- 查看token的长度

In [21]:
print(len(preprocessed))

4690


## 2.3 给token编号

- 通过如下的embedding层,我们可以给token编号

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/06.webp" width="500px">

- 我们要创建一个表格,给所有的token给映射到不同的标号上

In [22]:
all_words = sorted(set(preprocessed))#从去掉重复的字符
vocab_size = len(all_words)#计总的单词书

print(vocab_size)

1130


In [23]:
vocab = {token:integer for integer,token in enumerate(all_words)}##先把word进行编号,再按照单词或者标点为索引(有HashList那味道了)

- 看一下前50个是怎样的

In [25]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break ##遍历到前五十个

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


- 接下来,我们将通过一个短文本来感受下,处理后的效果是怎样的

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/07.webp?123" width="500px">

- 现在将所有内容整合到一个分词器类中。

In [30]:
class SimpleTokenizerV1:#一个实例的名字创立
    def __init__(self, vocab): ## 初始化一个字符串
        self.str_to_int = vocab #单词到整数的映射
        self.int_to_str = {i:s for s,i in vocab.items()} 
        #方便解码,进行整数到词汇的反向映射
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)##正则化分词标点符号
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()## 去掉两端空格与全部的空句
        ]
        ids = [self.str_to_int[s] for s in preprocessed]##整理完的额字符串列表对应到id,从字典出来
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids]) #映射整数id到字符串。join是用前面那个(“ ”)联结成一个完整的字符串
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text) #使用正则表达式，去除标点符号前的多余空格
        # \s+匹配一个或者多个空白  \1 替换到匹配
        return text

- `encode` 函数将文本转换为标记 ID。
- `decode` 函数将标记 ID 转换回文本。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/08.webp?123" width="500px">

- 我们可以使用分词器将文本编码（即分词）为数字。
- 然后，这些整数可以作为大语言模型（LLM）的输入，进行嵌入。

In [31]:
tokenizer = SimpleTokenizerV1(vocab) #用vocab创造一个实例

text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text) #按照这个例子里的encode函数处理text
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


- 把数字重新映射回文字

In [32]:
tokenizer.decode(ids)#按照这个例子里的decode函数处理text

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [33]:
tokenizer.decode(tokenizer.encode(text))#按照这个例子里的decode函数处理(#按照这个例子里的encode函数处理text)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

## 2.4 添加特殊标记

- 文本的结尾需要特别的符号来表明

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/09.webp?123" width="500px">

- 一些分词器使用特殊标记来帮助大语言模型（LLM）获取额外的上下文信息。
- 其中一些特殊标记包括：
  - `[BOS]`（序列开始）表示文本的开始。
  - `[EOS]`（序列结束）表示文本的结束（通常用于连接多个不相关的文本，例如两个不同的维基百科文章或两本不同的书籍等）。
  - `[PAD]`（填充）如果我们使用大于1的批次大小训练LLM（我们可能会包含不同长度的多篇文本），使用填充标记将较短的文本填充至最长的长度，以确保所有文本具有相同的长度。
- `[UNK]` 用于表示词汇表中没有的词。

- 请注意，GPT-2不需要上述提到的这些标记，它只使用 `<|endoftext|>` 标记。
- `<|endoftext|>` 类似于上述提到的 `[EOS]` 标记。
- GPT 还使用 `<|endoftext|>` 进行填充（因为我们在批量输入训练时通常使用掩码，所以无论这些填充标记是什么，都不会影响模型的训练，因为填充标记不会被关注）。
- GPT-2 不使用 `<UNK>` 标记来表示词汇表外的词；相反，GPT-2 使用字节对编码（BPE）分词器，将单词分解为子词单元，后续将进一步讨论这一点。

- 我们在两个独立文本之间使用 `<|endoftext|>` 标记：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/10.webp" width="500px">

- 看一下接下来会发生什么

In [37]:
tokenizer = SimpleTokenizerV1(vocab)  ##用vocab创造一个实例

text = "Hello, do you like tea. Is this-- a test?"

tokenizer.encode(text)

KeyError: 'Hello'

- 上述操作会产生一个错误，因为单词“Hello”不在词汇表中。
- 为了处理这种情况，我们可以向词汇表中添加类似 `"<|unk|>"` 的特殊标记，用于表示未知词汇。
- 因为我们已经在扩展词汇表，那么我们可以再添加一个标记 `"<|endoftext|>"`，该标记在GPT-2训练中用于表示文本的结束（它也用于连接的文本之间，例如当我们的训练数据集包含多篇文章、书籍等时）。

In [18]:
all_tokens = sorted(list(set(preprocessed)))#set去重 list把处理后的重新变为列表,然后排序
all_tokens.extend(["<|endoftext|>", "<|unk|>"])#加上未知的表示

vocab = {token:integer for integer,token in enumerate(all_tokens)}
#遍历 enumerate(all_tokens) 中的每个元组 (integer, token)，以 token 作为键，integer 作为值创建字典条目。

In [19]:
len(vocab.items())

1132

In [20]:
for i, item in enumerate(list(vocab.items())[-5:]):#输出后五个内容与其标号
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


- 因此增加 `<unk>`不失为一种好的选择

In [21]:
class SimpleTokenizerV2:##版本2.0,启动!
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}#s为单词,i是key
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)#正则化按照标点分类
        preprocessed = [item.strip() for item in preprocessed if item.strip()]#去掉两头与所有空余句
        preprocessed = [
            item if item in self.str_to_int 
            else "<|unk|>" for item in preprocessed
            #遍历 preprocessed 中的每个 item，如果 item 存在于 self.str_to_int（即词汇表）中，就保留 item
            #如果不存在（即该单词或符号未定义在词汇表中），就替换为特殊标记 <|unk|>。
            #拓展:推导式（如列表推导式）是一种紧凑的语法，专门用于生成新列表（或其他容器）
            #与普通 for 循环相比，它更加简洁和高效，但逻辑复杂时可能会降低可读性。
        ]

        ids = [self.str_to_int[s] for s in preprocessed]#单词或标点映射为整数列表
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [ ]:
- 用优化后的分词器对文本进行操作

In [22]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))#用句子分隔符链接两个句子

print(text)  #跟第一个一样,但不会报错了

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [23]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [24]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

## 2.5 字节对编码

- GPT-2 使用字节对编码（BPE）作为其分词器。
- 这种方式允许模型将不在预定义词汇表中的单词分解为更小的子词单元，甚至是单个字符，从而使其能够处理词汇表外的词汇。
- 例如，如果 GPT-2 的词汇表中没有“unfamiliarword”这个单词，它可能会将其分词为 ["unfam", "iliar", "word"]，或者根据训练好的 BPE 合并规则进行其他子词分解。
- 原始的 BPE 分词器可以在这里找到：[https://github.com/openai/gpt-2/blob/master/src/encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py)
- 在本章中，我们使用了 OpenAI 开源的 [tiktoken](https://github.com/openai/tiktoken) 库中的 BPE 分词器，该库在 Rust 中实现了核心算法，以提高计算性能。
- 在 [./bytepair_encoder](../02_bonus_bytepair-encoder) 中创建了一个笔记本，对比了这两种实现的效果（在样本文本上，tiktoken 的速度大约快了 5 倍）。

In [38]:
# pip install tiktoken

Note: you may need to restart the kernel to use updated packages.


In [42]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))#验证下载并输出版本信息

tiktoken version: 0.8.0


In [43]:
tokenizer = tiktoken.get_encoding("gpt2")#初始化GPT2!

In [44]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})#输出分词的id,可以允许endoftext

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [45]:
strings = tokenizer.decode(integers)
#按照数字解码回去

print(strings)                                                                  

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


- BPE tokenizers将未知词汇分解为子词和单个字符。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/11.webp" width="300px">

## 2.6 利用滑动窗口进行数据采样

- 现在我们训练的大语言模型（LLMs）时是一次生成一个单词，因此希望根据训练数据的要求进行准备，使得序列中的下一个单词作为预测目标。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/12.webp" width="400px">

In [30]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)#读入了一个text并编码到enc_text里面
print(len(enc_text))

5145


- 对于每个文本块，我们需要输入和目标。
- 由于我们希望模型预测下一个单词，因此我们要生成目标是将输入右移一个位置后的单词。

In [31]:
enc_sample = enc_text[50:]#从第五十一个开始向后

In [32]:
context_size = 4  #sliding windows4

x = enc_sample[:context_size]#开头四个
y = enc_sample[1:context_size+1]#第二个开始的四个

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


- 就像预言家一个晚上只能预言一个玩家,我们的模型一次也只能预测一个单词

In [33]:
for i in range(1, context_size+1):
    #文本成输入 context,先输出有什么,然后输出下一个是什么编号
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [34]:
for i in range(1, context_size+1):
    #文本成输入 context,先输出有什么,然后输出下一个是什么单词
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


- 我们将在后续章节中处理下一个单词预测，届时会介绍注意力机制。
- 目前，我们实现一个简单的数据加载器，它遍历输入数据集并返回右移一个位置后的输入和目标。

- 安装并导入 PyTorch（安装提示请参见附录A）。

In [48]:
import torch
print("PyTorch version:", torch.__version__)

PyTorch version: 2.5.1


- 用滑动窗口法运行,窗口位置每次加一

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/13.webp?123" width="500px">

- 创建数据集和数据加载器，从输入文本数据集中提取文本块。

In [49]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    #让GPT初始化一个类型
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})#id是文本内容编码过来的

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):#slidingwindows的诗钞
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [56]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer他都已经写了哈哈哈哈
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

- 让我们使用批次大小为1、上下文大小为4的设置，测试数据加载器在大语言模型（LLM）中的表现。

In [53]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [39]:
dataloader = create_dataloader_v1(#raw_text 中创建一个数据加载器 但是所批次
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)#数据加载器 dataloader 转换为一个迭代器
first_batch = next(data_iter)
print(first_batch)                                                                                        

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [40]:
second_batch = next(data_iter)
print(second_batch)                      

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


- 下面是一个示例，步幅等于上下文长度（此处为4）：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/14.webp" width="500px">

- 我们还可以批量输出。
- 因为过多的重叠可能导致过拟合,这里增加了步幅。

In [41]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## 2.7 创建标记嵌入(Creating token embeddings)

- 数据已经几乎准备好用于大语言模型（LLM）。
- 最后，让我们使用嵌入层将标记转换为连续的向量表示。
- 通常，这些嵌入层是大语言模型的一部分，并在模型训练过程中进行更新（训练）。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/15.webp" width="400px">

- 假设我们有以下四个输入示例，经过分词后对应的输入 ID 为 2、3、5 和 1：

In [42]:
input_ids = torch.tensor([2, 3, 5, 1])#要加入2,3,5,1的字符

- 为了简化问题，假设我们只有一个包含 6 个词的小型词汇表，并且我们希望创建大小为 3 的嵌入。

In [43]:
vocab_size = 6#嵌入层需要支持的唯一标记的总数
output_dim = 3#嵌入向量的维度

torch.manual_seed(123)#用于设置随机数生成器的种子，确保结果的可复现性
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)#每行表示一个标记的嵌入向量。

- 结果是个6*3的矩阵

In [44]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


- 对于熟悉（one-hot encoding）的人来说，上述嵌入层方法本质上只是实现one-hot编码后接矩阵乘法的更高效方式，具体实现可以参考[./embedding_vs_matmul](../03_bonus_embedding-vs-matmul)中的补充代码。
- 嵌入层只是对one-hot encoding和矩阵乘法方法的高效实现，因此可以将其视为一个神经网络层，并通过反向传播进行优化。

- 通过下列操作,我们可以吧id3映射到一个三纬的矩阵

In [45]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


- 请注意，上述内容是 `embedding_layer` 权重矩阵中的第四行。
- 要嵌入上述所有四个 `input_ids` 值，我们执行以下操作：

In [46]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


- 嵌入层本质上是一个查找操作：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/16.webp?123" width="500px">

- **您可能对比较嵌入层与常规线性层的附加内容感兴趣：[../03_bonus_embedding-vs-matmul](../03_bonus_embedding-vs-matmul)**

## 2.8 位置信息编码

- 嵌入层将 ID 转换为相同的向量表示

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/17.webp" width="400px">

- 位置信息与标记向量结合，形成大语言模型的最终输入：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/18.webp" width="500px">

- 字节对编码器的词汇表大小为 50,257：
- 假设我们想将输入标记编码为一个 256 维的向量表示：

In [47]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)#映射为tensor

- 如果我们从数据加载器中采样数据，我们将每个批次中的标记嵌入为一个 256 维的向量。
- 如果我们有一个批次大小为 8，每个批次包含 4 个标记，这将得到一个 8 x 4 x 256 的张量：

In [48]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [49]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [50]:
token_embeddings = token_embedding_layer(inputs)#调用token_embedding_layer将输入inputs映射为对应的嵌入向量。
print(token_embeddings.shape)

torch.Size([8, 4, 256])


- GPT-2 使用绝对位置嵌入，因此我们只需要创建另一个位置嵌入层：

In [51]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
#目的是为输入序列中的每个位置生成一个向量,表明位置信息

In [ ]:
- 嵌入层本质上是一个查找表,大小为(context_length, output_dim)

In [52]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))#生成一个连续整数的1D tensor

print(pos_embeddings.shape)

torch.Size([4, 256])


- 为了创建大语言模型（LLM）中使用的输入嵌入，我们只需将标记嵌入和位置嵌入相加：

In [53]:
input_embeddings = token_embeddings + pos_embeddings#特征是词语信息跟位置信息的结合
print(input_embeddings.shape)

torch.Size([8, 4, 256])


- 在输入处理流程的初始阶段，输入文本被分割为独立的标记。
- 在此分割后，这些标记根据预定义的单词表转换为标记 ID：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/19.webp" width="400px">

# 总结与收获

请参见 [./dataloader.ipynb](./dataloader.ipynb) 代码笔记本，这是我们在本章中实现的数据加载器的简洁版，并将在后续章节中用于训练 GPT 模型。

请参见 [./exercise-solutions.ipynb](./exercise-solutions.ipynb) 获取习题解答。